In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense, Concatenate
import functions as f

# Veriyi yükle
train = pd.read_parquet('train_final.parquet') 

In [ ]:
train = train.drop(['month'], axis=1)

In [ ]:
X_train_full = train.drop(['target'], axis=1)
y_train_full = train['target']

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.1, random_state=42)

X_train_ids = X_train['id']
X_valid_ids = X_valid['id']

cat_train = X_train[['carrier', 'devicebrand']]
cat_train = pd.get_dummies(cat_train)

cat_valid = X_valid[['carrier', 'devicebrand']]
cat_valid = pd.get_dummies(cat_valid)

X_train = X_train.drop(['id','carrier', 'devicebrand'], axis=1)
X_valid = X_valid.drop(['id','carrier', 'devicebrand'], axis=1)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit([['1', '2', '3','4', '5', '6','7', '8', '9']])

In [ ]:
y_train = mlb.transform(y_train)
y_valid = mlb.transform(y_valid)

In [ ]:
import tensorflow as tf

class DLRM(tf.keras.Model):
    def __init__(self, num_dense_features, num_sparse_features, embedding_dim):
        super(DLRM, self).__init__()
        # Sürekli özellikler için çok katmanlı algılayıcı (MLP)
        self.dense_mlp = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu')
        ])
        
        # Kategorik özellikler için embedding tabloları
        self.embeddings = [tf.keras.layers.Embedding(100, embedding_dim) for _ in range(num_sparse_features)]
        
        # Üst MLP
        self.top_mlp = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(9, activation='softmax')
        ])
    
    def call(self, inputs):
        dense_x, sparse_x = inputs
        # Sürekli özellikleri işle
        dense_out = self.dense_mlp(dense_x)
        
        # Kategorik özellikleri işle
        sparse_out = [emb(sparse_x[:, i]) for i, emb in enumerate(self.embeddings)]
        sparse_out = tf.concat(sparse_out, axis=1)
        
        # Sürekli ve kategorik özellikleri birleştir
        x = tf.concat([dense_out, sparse_out], axis=1)
        
        # Üst MLP
        out = self.top_mlp(x)
        
        return out


In [ ]:
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

num_dense_features = len(X_train.columns) #53
num_sparse_features = len(cat_train.columns) #620
embedding_dim = 31


# Modeli oluşturun
model = DLRM(num_dense_features, num_sparse_features, embedding_dim)

# Modeli derleyin
model.compile(optimizer=Adam(), loss=BinaryCrossentropy())

# Modeli eğitin
model.fit([X_train, cat_train], [y_train], epochs=10, batch_size=32)



In [ ]:
model.save("model_saved", save_format="tf")


In [ ]:
cat_valid

In [ ]:
# Eksik sütunları bulun
missing_cols = set(cat_train.columns) - set(cat_valid.columns)

# Eksik sütunları 'cat_valid' DataFrame'ine ekleyin
for c in missing_cols:
    cat_valid[c] = 0

# Sütunları 'cat_train' ile aynı sırayla düzenleyin
cat_valid = cat_valid[cat_train.columns]


In [ ]:


# Modelin performansını değerlendirin
y_pred = model.predict([X_valid, cat_valid])



In [ ]:
y_pred[5]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

class_names = ['1', '2', '3', '4', '5', '6', '7', '8', '9']

# En yüksek 3 olasılığa sahip sınıfların indekslerini bulun
top3_classes = np.argsort(y_pred, axis=1)[:, -3:]

# Indeksleri sınıf isimleriyle değiştirin
top3_class_names = [[class_names[i] for i in indices] for indices in top3_classes]

# MultiLabelBinarizer'ı oluşturun
mlb = MultiLabelBinarizer()
mlb.fit([['1', '2', '3', '4', '5', '6', '7', '8', '9']])

# Tahminleri çoklu etiketli bir biçime dönüştürün
y_pred_transformed = mlb.transform(top3_class_names)


In [ ]:
y_pred_transformed[0]

In [ ]:
jaccard = f.jaccard_similarity_score(y_valid, y_pred_transformed)
jaccard

In [ ]:
# Veriyi yükle
Test = pd.read_parquet('test_final.parquet') 

In [ ]:
Test

In [ ]:
Test = Test.drop(['month'], axis=1)

In [ ]:
Test_ids = Test['id']

In [ ]:
Test = Test.drop(['id'], axis=1)

In [ ]:
cat_Test = Test[['carrier', 'devicebrand']]
cat_Test = pd.get_dummies(cat_Test)

In [ ]:
Test = Test.drop(['carrier', 'devicebrand'], axis=1)

In [ ]:
# Eksik sütunları bulun
missing_cols = set(cat_train.columns) - set(cat_Test.columns)

# Eksik sütunları 'cat_valid' DataFrame'ine ekleyin
for c in missing_cols:
    cat_Test[c] = 0

# Sütunları 'cat_train' ile aynı sırayla düzenleyin
cat_Test = cat_Test[cat_train.columns]

In [ ]:
Test_y_pred = model.predict([Test, cat_Test])

In [ ]:
# Veriyi yükle
ssf = pd.read_parquet('submission_sample_final.parquet') 

In [ ]:
Test_y_pred[0]

In [ ]:
# En yüksek 3 olasılığa sahip sınıfların indekslerini bulun
top3_classes = np.argsort(Test_y_pred, axis=1)[:, -3:]

# Indeksleri sınıf isimleriyle değiştirin
top3_class_names = [[class_names[i] for i in indices] for indices in top3_classes]

# MultiLabelBinarizer'ı oluşturun
mlb = MultiLabelBinarizer()
mlb.fit([['1', '2', '3', '4', '5', '6', '7', '8', '9']])

# Tahminleri çoklu etiketli bir biçime dönüştürün
Test_y_pred_transformed = mlb.transform(top3_class_names)


In [ ]:
Test_y_pred_transformed[0]

In [ ]:
# Her bir NumPy dizisini stringe dönüştürün
str_list = [''.join(map(str, arr)) for arr in Test_y_pred_transformed]

str_list  # Çıktı: ['010001010', '000101010']

In [ ]:
# Tahminleri içeren yeni bir veri çerçevesi oluşturun
tahmin_verisi = pd.DataFrame({'id': Test_ids, 'target': str_list})

# Tahmin verisini CSV dosyasına kaydedin
tahmin_verisi.to_csv('ilk_dlrm.csv', index=False, columns=['id', 'target'])

In [ ]:
tahmin_verisi